In [7]:
!pip install chromadb-client sentence_transformers
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.7/224.7 kB 4.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [16]:
from numpy import dot
from numpy.linalg import norm
from sentence_transformers import SentenceTransformer
from openai import OpenAI
import json
import pandas as pd
import chromadb
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
gpt = OpenAI(api_key='sk-proj-SyXBjT4PPB98njKb7GYFT3BlbkFJg3ItqCdr7WzBe7zHTr6b')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [40]:
class EmbeddingModel:
    def __init__(self, model):
        self.model = model

    def __call__(self, input):
        embeddings = []
        for text in input:
            if ' -- ' not in text:
                raise ValueError("Input text does not contain expected '--' delimiter.")

            question = text.split(' -- ')[0]

            embedded = self.model.encode(question, show_progress_bar=False)
            embeddings.append(embedded.tolist())

        return embeddings

embedding_model = EmbeddingModel(model)


In [84]:
path_to_dataset = '/content/dataset.json'

with open(path_to_dataset, 'r', encoding='utf-8') as file:
    data = json.load(file)

data = data['data']

df = pd.DataFrame(columns=['title', 'url', 'source', 'business_line_id', 'direction', 'product', 'type', 'description'])
for dict_ in data:
    df.loc[df.shape[0], :] = [dict_[col_name] for col_name in df.columns]



,title,url,source,business_line_id,direction,product,type,description
0,Как посмотреть лимиты счета?,https://www.tinkoff.ru/business/help/business-...,web,business,business-payout,jump,card,"Вы можете посмотреть лимиты по счету на месяц,..."
1,Как посмотреть выплаты за определенный период?,https://www.tinkoff.ru/business/help/business-...,web,business,business-payout,jump,card,Это можно сделать в личном кабинете Jump.Finan...
2,Как посмотреть список выплат с ошибкой?,https://www.tinkoff.ru/business/help/business-...,web,business,business-payout,jump,card,Это можно сделать в личном кабинете Jump.Finan...
3,"Как посмотреть список выплат, которые ждут под...",https://www.tinkoff.ru/business/help/business-...,web,business,business-payout,jump,card,Это можно сделать в личном кабинете Jump.Finan...
4,Как посмотреть лимиты по филиалам?,https://www.tinkoff.ru/business/help/business-...,web,business,business-payout,jump,card,В разделе «Сводка» вы можете отслеживать лимит...
...,...,...,...,...,...,...,...,...
3232,Как получать зарплату на карту Тинькофф,https://www.tinkoff.ru/business/help/business-...,web,business,bank-faq-crisis,,article,Почему выгодно получать зарплату на карту Тинь...
3233,Как сервисы Тинькофф Бизнеса работают в нового...,https://www.tinkoff.ru/business/help/happy-hol...,web,business,business,holidays,article,Служба поддержки График обслуживания клиентов ...
3234,Клиентам Тинькофф Бизнеса: актуальные вопросы,https://www.tinkoff.ru/business/help/faq-crisis/,web,business,conditions,faq-crisis,article,Валютные операции Какие валютные операции пров...
3235,Подключиться к Тинькофф,https://www.tinkoff.ru/business/help/business-...,web,business,bank-faq-crisis,,article,"Выберите юрлицо, в котором работаете Найти нуж..."


In [92]:
df.iloc[0, :]['title']

'Как посмотреть лимиты счета?'

In [95]:
questions_original = [f"{df.iloc[i, :]['title']} -- {df.iloc[i, :]['description']} -- {df.iloc[i, :]['url']}" for i in range(len(df)) ]
embeddings = pd.read_csv('/content/original_embeddings.csv')
embeddings_original = [[float(x) for x in embeddings.iloc[i, :]] for i in range(len(embeddings))]
ids = [f'id{i}' for i in range(len(embeddings))]
len(embeddings_original) == len(questions_original)

In [109]:

client = chromadb.HttpClient(host='51.250.103.4', port=8000)
collection = client.create_collection(name="tinkoff_collection", embedding_function=embedding_model, metadata={"hnsw:space": "cosine"})

In [115]:
collection.add(
    documents = questions_original,
    embeddings = embeddings_original,
    ids = ids
)

In [119]:

df2 = pd.read_csv('/content/df_augmented.csv')
questions_augmentated = [f"{df2.iloc[i, :]['title']} -- {df2.iloc[i, :]['description']} -- {df2.iloc[i, :]['url']}" for i in range(len(df)) ]
embeddings2 = pd.read_csv('/content/augmented_embedding.csv')
embeddings_augmentated = [[float(x) for x in embeddings2.iloc[i, :]] for i in range(len(embeddings2))]
len(questions_augmentated) == len(embeddings_augmentated)
ids2 = [f'id{i}' for i in range(len(embeddings), len(embeddings) + len(embeddings_augmentated))]
len(ids2)

In [125]:
collection.add(
    documents = questions_augmentated,
    embeddings = embeddings_augmentated,
    ids = ids2
)

In [ ]:
# collection = client.get_collection(name="test") # Get a collection object from an existing collection, by name. Will raise an exception if it's not found.
# collection = client.get_or_create_collection(name="test") # Get a collection object from an existing collection, by name. If it doesn't exist, create it.
# client.delete_collection(name="my_collection")

In [127]:
results = collection.query(
    query_texts=["Как взять кредит? -- "],
    n_results=2
)
print(results)

{'ids': [['id4311', 'id1074']], 'distances': [[0.027070283889770508, 0.05809366703033447]], 'embeddings': None, 'metadatas': [[None, None]], 'documents': [['Как получить кредит? -- В личном кабинете ИП нажмите «Добавить продукт» и выберите «Оформить кредит». На странице кредита заполните форму заявки: укажите желаемую сумму, срок и ежемесячный доход. Заявку может отправить только сам ИП, если у него еще не оформлен этот кредит. Банк рассматривает заявку 1\u2060—\u20602 дня. В это время вам могут позвонить из банка и уточнить условия кредита. Может понадобиться встреча с представителем банка, в таком случае представитель приедет к вам в удобное время. Если банк одобрит кредит, в личном кабинете появится новый счет «Кредит наличными». -- https://www.tinkoff.ru/business/help/loans/cashloan/about/how-to-get-cag/?card=get-cag', 'Как оформить кредит? -- В личном кабинете ИП нажмите «Добавить продукт» и выберите «Оформить кредит». На странице кредита заполните форму заявки: укажите желаемую с